# Lab 5: Species Distribution Modeling with Bagging 
Climate change and deforestation are accelerating ecosystem degradation, pushing animal species toward the brink of extinction. Understanding the distribution of animals and the factors that influence their ability to thrive in different environments is critical for conservation efforts. By studying these relationships, biological conservationists can develop informed strategies to protect endangered species and maintain biodiversity across diverse ecosystems.

Species are defined by their behavioral, physiological, and ecological attributes, which shape their roles in ecosystems. In turn, ecosystems and their functions are deeply influenced by the species that inhabit them. This reciprocal relationship makes evaluating species' functional diversity and distributions essential for research in biogeography, community ecology, macroevolution, and conservation. Functional diversity is determined by traits such as diet, foraging strata, trophic level, activity cycle, litter size, generation length, habitat breadth, and body mass. These traits influence an animal’s ability to survive and adapt to different climates, playing a crucial role in ecosystem stability.

In this lab, we will explore the connections between animal behaviors, functional traits, and species distributions across different climates.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.inspection import permutation_importance
from sklearn.utils import resample

### Step 1: Standardizing column names

The dataset contains several columns that describe different aspects of bird diets. These column names currently include spaces, colons (:), and special characters, which can make them difficult to work with in code.

Load the dataset (BirdTraitData.csv) into a Pandas DataFrame.

The 7 column names that start with "Diet:" should be renamed to simpler version starting with `Diet_` in a way that conforms with this list of key variables:

`AdultBodyMass`, `DietBreadth`, `Diet_Invertebrates`, `Diet_Vertebrates`,`Diet_Fruits`, `Diet_Flowers`, `Diet_Seeds`, `Diet_Plants`, `Diet_Other`, `TrophicLevel`,`ActivityCycle`, `ForagingStratum`, `MigratoryStatus`,`IncubationTime`, `ClutchSize`, `EggLength`, `EggWidth`,`EggMass`, `NestingHabit`,`MaxLongevity`, `Subarid`

Then select only that list of 21 key variables. **Print the head of your dataframe.** 

In [26]:
birds = pd.read_csv('data/BirdTraitData.csv', encoding='unicode_escape')
birds

,Order,Family,Genus,Species,Authority,CommonName,AdultBodyMass,DietBreadth,Diet: invertebrates,Diet: vertebrates,...,Sofia,Vakinankaratra,Vatovavy.Fitovinany,Antananarivo,Antsiranana,Fianarantsoa,Mahajanga,Toamasina,Toliara,References
0,Accipitriformes,Accipitridae,Accipiter,Accipiter francesiae,"Smith, 1834",Frances's Sparrowhawk,138.5,2,yes,yes,...,1,1,1,1,1,1,1,1,1,109; 254; 168; 376; 368; 199
1,Accipitriformes,Accipitridae,Accipiter,Accipiter henstii,"(Schlegel, 1873)",Henst's Goshawk,1050.0,1,no,yes,...,1,1,1,1,1,1,1,1,1,109; 254; 376; 200; 168
2,Accipitriformes,Accipitridae,Accipiter,Accipiter madagascariensis,"Smith, 1834",Madagascar Sparrowhawk,181.0,2,yes,yes,...,1,1,1,1,1,1,1,1,1,109; 254; 168; 376; 202
3,Accipitriformes,Accipitridae,Aviceda,Aviceda madagascariensis,"(Smith, 1834)",Madagascar Baza,292.0,2,yes,yes,...,1,1,1,1,1,1,1,1,1,376; 331; 376; 168; 254; 195
4,Accipitriformes,Accipitridae,Buteo,Buteo brachypterus,"Hartlaub, 1860",Madagascar Buzzard,511.0,2,yes,yes,...,1,1,1,1,1,1,1,1,1,109; 168; 254; 376; 198; 186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,Suliformes,Fregatidae,Fregata,Fregata ariel,"(Gray, 1845)",Lesser Frigatebird,806.0,3,yes,yes,...,1,0,0,0,1,0,1,1,0,109; 254; 376; 268; 168; 219; 255; 4
238,Suliformes,Fregatidae,Fregata,Fregata minor,"(Gmelin, 1789)",Greater Frigatebird,1362.0,3,yes,yes,...,1,0,0,0,1,0,1,1,1,109; 254; 376; 270; 168; 4; 219
239,Suliformes,Phalacrocoracidae,Microcarbo,Microcarbo africanus,"(Gmelin, 1789)",Long tailed Cormorant,545.0,2,yes,yes,...,1,1,0,1,1,1,1,1,1,109; 254; 376; 267; 168; 219
240,Suliformes,Sulidae,Sula,Sula leucogaster,"(Boddaert, 1783)",Brown booby,1219.0,2,yes,yes,...,1,0,1,0,1,1,1,1,1,109; 254; 376; 45; 168; 4; 219


In [27]:
birds.columns = birds.columns.str.replace(':', '_').str.replace(" ", "")
birds.columns = birds.columns.str.title()
birds.columns

Index(['Order', 'Family', 'Genus', 'Species', 'Authority', 'Commonname',
       'Adultbodymass', 'Dietbreadth', 'Diet_Invertebrates',
       'Diet_Vertebrates', 'Diet_Fruits', 'Diet_Flower/Nectar/Pollen/Gums',
       'Diet_Seeds', 'Diet_Otherplantmaterials',
       'Diet_Scavenge;Garbage;Carrion;Offal;Carcasses', 'Trophiclevel',
       'Activitycycle', 'Foragingstratum', 'Migratorystatus', 'Incubationtime',
       'Clutchsize', 'Egglength', 'Eggwidth', 'Eggmass', 'Nestinghabit',
       'Maxlongevity', 'Dry', 'Humid', 'Montane', 'Subarid', 'Subhumid',
       'Alaotra.Mangoro', 'Amoron.I.Mania', 'Analamanga', 'Analanjirofo',
       'Androy', 'Anosy', 'Atsimo.Andrefana', 'Atsimo.Atsinanana',
       'Atsinanana', 'Betsiboka', 'Boeny', 'Bongolava', 'Diana',
       'Haute.Matsiatra', 'Ihorombe', 'Itasy', 'Melaky', 'Menabe', 'Sava',
       'Sofia', 'Vakinankaratra', 'Vatovavy.Fitovinany', 'Antananarivo',
       'Antsiranana', 'Fianarantsoa', 'Mahajanga', 'Toamasina', 'Toliara',
       'Refere

`AdultBodyMass`, `DietBreadth`, `Diet_Invertebrates`, `Diet_Vertebrates`,`Diet_Fruits`, `Diet_Flowers`, `Diet_Seeds`, `Diet_Plants`, `Diet_Other`, `TrophicLevel`,`ActivityCycle`, `ForagingStratum`, `MigratoryStatus`,`IncubationTime`, `ClutchSize`, `EggLength`, `EggWidth`,`EggMass`, `NestingHabit`,`MaxLongevity`, `Subarid`

In [33]:

birds = birds.rename(columns = {'Diet_Otherplantmaterials':'Diet_Plants', 'Diet_Scavenge;Garbage;Carrion;Offal;Carcasses':'Diet_Other'})
birds = birds.rename(columns = {'Adultbodymass':'AdultBodyMass', 'Dietbreadth':'DietBreadth', 'Diet_Flower/Nectar/Pollen/Gums':'Diet_Flowers', 'Trophiclevel':'TrophicLevel', 'Activitycycle':'ActivityCycle', 'Foragingstratum':'ForagingStratum', 'Migratorystatus':'MigratoryStatus', 'Incubationtime':'IncubationTime', 'Clutchsize':'ClutchSize', 'Egglength':'EggLength', 'Eggwidth':'EggWidth', 'Nestinghabit':'NestingHabit', 'Maxlongevity':'MagLongevity'})
birds.columns

Index(['Order', 'Family', 'Genus', 'Species', 'Authority', 'Commonname',
       'AdultBodyMass', 'DietBreadth', 'Diet_Invertebrates',
       'Diet_Vertebrates', 'Diet_Fruits', 'Diet_Flowers', 'Diet_Seeds',
       'Diet_Plants', 'Diet_Other', 'TrophicLevel', 'ActivityCycle',
       'ForagingStratum', 'MigratoryStatus', 'IncubationTime', 'ClutchSize',
       'EggLength', 'EggWidth', 'Eggmass', 'NestingHabit', 'MagLongevity',
       'Dry', 'Humid', 'Montane', 'Subarid', 'Subhumid', 'Alaotra.Mangoro',
       'Amoron.I.Mania', 'Analamanga', 'Analanjirofo', 'Androy', 'Anosy',
       'Atsimo.Andrefana', 'Atsimo.Atsinanana', 'Atsinanana', 'Betsiboka',
       'Boeny', 'Bongolava', 'Diana', 'Haute.Matsiatra', 'Ihorombe', 'Itasy',
       'Melaky', 'Menabe', 'Sava', 'Sofia', 'Vakinankaratra',
       'Vatovavy.Fitovinany', 'Antananarivo', 'Antsiranana', 'Fianarantsoa',
       'Mahajanga', 'Toamasina', 'Toliara', 'References'],
      dtype='object')

### Step 2: Encode categorical variables
In our dataset, some columns contain categorical (non-numeric) data, such as species names or habitat types that need to be converted to numerical representations. Let's use label encoding to assign a unique number to each category in a column.

Encode the data using this process:
1. Find all categorical columns in df using `.select_dtypes(include=['object'])`
2. Loop through each categorical column and apply `LabelEncoder()`
3. Replace the original categorical columns with their encoded values.
4. Print the head of your dataframe to ensure that labels were in fact encoded. 

### Step 3: Defining features, splitting data, and training a bagging classifier
Our goal for this analysis is to predict whether a given bird species is present in Subarid climate zones. 

1. Drop the corresponding variable from the features (X) and define it as the target (y). Then perform the train-test split with 70/30 train/test and a random state of 808.

2. Initialize a bagging classifier with 100 trees, `oob_score =True`, `bootstrap = True` and `random_state = 808`.  

3. Train the model.

### Step 4: Make predictions and evaluate the model

1. Generate predictions
- Use the trained bagging model to predict values for the test set (`X_test`).

2. Calculate the Out-of-Bag (OOB) Score
- The OOB score is an internal validation score computed using samples not included in bootstrapped subsets.

3. Compute model accuracy
- Print the OOB score and the bagging accuracy score.

### Step 5: Calculate and plot ROC curve

- Use `label=f"Bagging (AUC = {roc_auc_bagging:.2f})"` to include the AUC value in the plot.

Now how does the ROC curve look? How does this compare to our classification accuracy metric?  What might be going on? 

*your anwer here*

### Step 6: Introducing upsampling

The ROC curve indicates that our model is not performing well.  This is a common issue in imbalanced datasets, where one class significantly outnumbers the other. To improve our model’s ability to correctly classify both groups, we will upsample the minority class so that both classes have equal representation in the training set.

**Perform upsampling to balance the dataset**
1. Separate the majority and minority classes:
- Identify which class (0,1) is underrepresented (the minority class) and which is overrepresented (the majority class). 
- Separate both the `X_train` data and `y_train` data into all majority class and all minority class sets. 
2. Apply upsampling:
- Use `resample()` to create additional synthetic samples of the minority class *with replacement* until it is the same size as the majority class data. Use a random state of 808. 
3. Combine the balanced classes:
Merge the original majority class with the newly upsampled minority class.
4. Shuffle the resampled dataset:
- Randomly shuffle the data to ensure the model does not learn any unintended patterns from the order of the data. Use a random state of 808 for shuffling both the resampled X and y training data. 
5. Print the value counts for the resampled y training data to ensure that the data is balanced. 


In [1]:
# Separate majority and minority classes


# Upsample the minority class

# Combine majority and upsampled minority class

# Shuffle the dataset

# Print value counts 


### Step 7: Retrain and evaluate model using balanced data

Now that we have addressed the class imbalance by upsampling the minority class, we will retrain the bagging classifier on the newly balanced dataset and evaluate its performance. This will help us determine whether handling class imbalance improves model accuracy and its ability to distinguish between classes. Create a bagging model using your balanced data. Use a random state of 808. Print the accuracy and and ROC Curve for this new model with balanced data. 

How did this second model, trained on balanced data, do in comparison to the first on the accuracy and AUC metrics.  How should we interpret those results?  Did the upscaling prove useful?

*your answer here*

### Step 8: Analyzing feature importance with permutation importance

Understanding which bird traits contribute the most to our model’s predictions is crucial for interpreting results. We covered in lecture how the results of ensemble methods are harder to interpret than a single decision tree.  But we do have some tools to do it.  In this step, we will compute permutation importance, which measures the impact of each trait by shuffling its values and observing the effect on model performance. This will help us identify which functional traits—such as diet, body mass, or habitat breadth—play the biggest role in predicting whether a species is found in a subarid environment.

Use `permutation_importance()` to calculate the importance values.  You'll need to pass it the model, the feature and target test data, and an n_repeats value of 10.

What features are most important in our model of species distribution?

*your answer here*